##**Librerias y dependencias necesarias**


In [10]:
from google.colab import drive #Acceso Drive
import os #Acceder a directorios
import pandas as pd #manejo de dataframes
import folium #mostrar mapa con geocolocacion 
import numpy as np #Numerical Python
import pickle #abrir datos en .pk

##**Acceso al drive y ubicacion archivos**

In [11]:
#Damos acceso a colab a nuestro Drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [12]:
#Nos ubicamos en la direccion de interes
#Alli estaran los data frames y el .pk y lo mapas.html generados
ruta_principal='/content/drive/MyDrive/Bio_Acustica' #direccion raiz de nuestro proyecto actual
os.chdir(ruta_principal) #Ubicarnos en el directorio
print("Ubicados en :",ruta_principal)
!ls

Ubicados en : /content/drive/MyDrive/Bio_Acustica
BioAcustica.ipynb	       mapa_grabadoras_cluster3.html
ClusterDispersion.ipynb        mapa_grabadoras_cluster_5c_1.html
ClusterDispersionV2.ipynb      mapa_grabadoras_cluster_5c_2.html
data_5c.pkl		       mapa_grabadoras_cluster_5c_3.html
data.pkl		       mapa_grabadoras_cluster_5c_4.html
df_audios_cluster.csv	       mapa_grabadoras_cluster_5c_5.html
df_audios.csv		       mapa_grabadoras_cluster_5c.html
df_grabadoras.csv	       mapa_grabadoras_cluster.html
mapa_grabadoras_cluster1.html  mapa_grabadoras.html
mapa_grabadoras_cluster2.html  mapa.html


#**Cargar Data**

In [13]:
#Cargamos df con informacion de las 31 grabadoras
df_grabadoras = pd.read_csv('/content/drive/MyDrive/Bio_Acustica/df_grabadoras.csv')

#cargamos los audios separados segun su cluster (3 clusters)
with open('/content/drive/MyDrive/Bio_Acustica/data.pkl', 'rb') as f:
    pk_clusters = pickle.load(f)

#cargamos los audios separados segun su cluster (5 clusters)
with open('/content/drive/MyDrive/Bio_Acustica/data_5c.pkl', 'rb') as f:
    pk_clusters_5c = pickle.load(f)

In [ ]:
df_grabadoras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Recorder           31 non-null     object 
 1   Municipio          31 non-null     object 
 2   Vereda             31 non-null     object 
 3   Latitud            31 non-null     float64
 4   Longitud           31 non-null     float64
 5   Elevacion          31 non-null     int64  
 6   Habitat            31 non-null     object 
 7   Total_grabaciones  31 non-null     int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 2.1+ KB


In [ ]:
df_grabadoras.head()

,Recorder,Municipio,Vereda,Latitud,Longitud,Elevacion,Habitat,Total_grabaciones
0,G03,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,565
1,G04,San Roque,Playa Rica,6.4286,-75.0281,1310,Forest,418
2,G06,Alejandria,El Respaldo,6.4088,-75.0453,1302,Forest,887
3,G07,San Roque,El Nucito,6.4287,-75.0368,1296,Non- forest,621
4,G08,Alejandria,El Respaldo,6.4061,-75.0534,1314,Non- forest,648


In [ ]:
print("Numero de clusters",np.shape(pk_clusters)[0])
print("Numero de audios cluster 1",np.shape(pk_clusters[0])[0])
print("Numero de audios cluster 2",np.shape(pk_clusters[1])[0])
print("Numero de audios cluster 3",np.shape(pk_clusters[2])[0])
print("Forma de los datos",pk_clusters[0][0])

Numero de clusters 3
Numero de audios cluster 1 4759
Numero de audios cluster 2 977
Numero de audios cluster 3 264
Forma de los datos /media/mirp_ai/DATA1/Jaguas_2018/G52_m/G52_20180623_123200.wav


/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [ ]:
print("Numero de clusters",np.shape(pk_clusters_5c)[0])
print("Numero de audios cluster 1",np.shape(pk_clusters_5c[0])[0])
print("Numero de audios cluster 2",np.shape(pk_clusters_5c[1])[0])
print("Numero de audios cluster 3",np.shape(pk_clusters_5c[2])[0])
print("Numero de audios cluster 4",np.shape(pk_clusters_5c[3])[0])
print("Numero de audios cluster 5",np.shape(pk_clusters_5c[4])[0])
print("Forma de los datos",pk_clusters_5c[0][0])

Numero de clusters 5
Numero de audios cluster 1 720
Numero de audios cluster 2 2380
Numero de audios cluster 3 244
Numero de audios cluster 4 2519
Numero de audios cluster 5 137
Forma de los datos /media/mirp_ai/DATA1/Jaguas_2018/G28_m/G28_20180609_160000.wav


#Clusterizando

In [21]:
#Funcion que crea un data frame, con los datos de los clusters, para cada grabadora
#Argumentos: -df base con la informacion de las grabadoras -.pk con la informacion de los clusters
def create_df_cluster(df_grabadoras,pk_clusters,inf=0,sup=240000):
  df_grabadoras_cluster=df_grabadoras.copy()
  num_clusters = np.shape(pk_clusters)[0]
  for i in range(num_clusters):
    df_grabadoras_cluster[f"Cluster{i+1}"] = 0
    num_audios_cluster = np.shape(pk_clusters[i])[0]
    for j in range(num_audios_cluster):
        nom_aud=os.path.basename(pk_clusters[i][j])
        nom_grab=nom_aud.split("_")[0]
        hora_aud=nom_aud.split("_")[2]
        hora_aud=int(hora_aud.split(".")[0])
        if (hora_aud >= inf and hora_aud < sup ):
          df_grabadoras_cluster.loc[df_grabadoras_cluster["Recorder"] == nom_grab, "Cluster"+str(i+1)] += 1
  return df_grabadoras_cluster

In [16]:
nom_aud=os.path.basename(pk_clusters[0][0])
hora_aud=nom_aud.split("_")[2]
hora_aud=int(hora_aud.split(".")[0])
hora_aud

123200

In [28]:
#Llamamos funcion y el retorno se almacena en nuevo df 
df_grabadoras_cluster1=create_df_cluster(df_grabadoras,pk_clusters_5c,0,100000)

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [29]:
df_grabadoras_cluster1

,Recorder,Municipio,Vereda,Latitud,Longitud,Elevacion,Habitat,Total_grabaciones,Cluster1,Cluster2,Cluster3,Cluster4,Cluster5
0,G03,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,565,0,72,0,8,0
1,G04,San Roque,Playa Rica,6.4286,-75.0281,1310,Forest,418,0,14,10,11,0
2,G06,Alejandria,El Respaldo,6.4088,-75.0453,1302,Forest,887,0,75,6,14,0
3,G07,San Roque,El Nucito,6.4287,-75.0368,1296,Non- forest,621,0,48,5,37,0
4,G08,Alejandria,El Respaldo,6.4061,-75.0534,1314,Non- forest,648,0,92,0,18,0
5,G09,Santo Domingo,Nusito-Dantas,6.4204,-75.0474,1346,Forest,908,0,75,15,30,0
6,G13,Alejandria,El Respaldo,6.3968,-75.0186,1349,Forest,636,0,68,0,12,0
7,G15,San Roque,Playa Rica,6.4045,-75.0143,1325,Forest,874,0,120,5,5,0
8,G17,Alejandria,El Respaldo,6.3978,-75.0444,1389,Non- forest,870,2,40,0,30,3
9,G19,Alejandria,El Respaldo,6.4075,-75.0308,1376,Non- forest,886,0,98,15,17,0


## Distribucion en mapa

In [30]:
# Crear un diccionario para asignar colores a cada cluster
colores_clusters = {
    "Cluster1": 'blue',
    "Cluster2": 'green',
    "Cluster3": 'red',
    "Cluster4": 'orange',
    "Cluster5": 'black',
}
# Funcion que retorna un mapa de folium, con la distribucion de los clusters
# Argumentos: -df con informacion de las grabadoras incluyendo los clusters, -diccionario de colores para cada cluster, -cluster a graficar
def create_map_cluster(df_grabadoras_cluster,colores_clusters,cluster,marker=True):
  num_cluster=len(cluster)
  # Coordenadas del punto
  lat = 6.3867
  lng = -75.0276
  # Crear mapa centrado en las coordenadas
  m = folium.Map(location=[lat, lng], zoom_start=12.7)
  for i in df_grabadoras_cluster.index:
    text=" "
    lat,lng=df_grabadoras_cluster["Latitud"][i],df_grabadoras_cluster["Longitud"][i]
    for j in range(len(cluster)):
      if df_grabadoras_cluster.iloc[i, j+8]>0 and cluster[j]!=0 :
        radio= int(df_grabadoras_cluster.iloc[i, j+8]/6000*18000)
        folium.Circle(location=[lat, lng], radius=radio, color=colores_clusters[df_grabadoras_cluster.columns[j+8]], fill=True, fill_opacity=0.09,weight=1.2).add_to(m)
        text=text+df_grabadoras_cluster.columns[j+8]+":"+ str(df_grabadoras_cluster.iloc[i, j+8])+" "
    if marker==True:
      icon = folium.Icon(icon='glyphicon-map-marker', prefix='glyphicon', icon_color='black', icon_size=(9, 9))
      popup_text=df_grabadoras_cluster["Recorder"][i],"Lat:",df_grabadoras_cluster["Latitud"][i],"Lng:",df_grabadoras_cluster["Longitud"][i],"Elev:",df_grabadoras_cluster["Elevacion"][i] ,df_grabadoras_cluster["Habitat"][i],"Grabaciones:",df_grabadoras_cluster["Total_grabaciones"][i],text
      folium.Marker(location=[lat, lng], popup=popup_text, icon=icon, opacity=0.3).add_to(m)
  return m

In [31]:
create_map_cluster(df_grabadoras_cluster1,colores_clusters,cluster=[1,1,1,1,1])

In [ ]:
#Llamar funcion de graficar cluster en mapa para almacenarlo
create_map_cluster(df_grabadoras_cluster1,colores_clusters,cluster=[1,1,1]).save("/content/drive/MyDrive/Bio_Acustica/mapa.html")